In [ ]:
import math
import torch
import numpy as np
import matplotlib.pyplot as plt

device = "cpu"

In [ ]:
num_samples = 3000
num_classes = 3

# Generate random data

def p(t, f=0):
    x = (1-t)*torch.vstack([
            torch.cos(2*torch.pi*t + f),
            torch.sin(2*torch.pi*t + f)
        ]) + torch.randn(2, len(t))*0.03
    return torch.transpose(x, 0, 1)

X, Y = [], []
a = 2*torch.pi/num_classes
samples_per_class = num_samples//num_classes
for c in range(num_classes):
    t = torch.linspace(0, 0.9, samples_per_class)
    X.append( p(t, c*a) )
    # y is one hot encoded
    y = torch.zeros((samples_per_class, num_classes), dtype=torch.float32)
    y[:, c] = 1
    Y.append(y)
X = torch.vstack(X)
Y = torch.vstack(Y)
# randomize 3 times X and Y
for i in range(3):
    idx = torch.randperm(num_samples)
    X = X[idx]
    Y = Y[idx]

plt.figure()
for c in range(num_classes):
    idx = torch.where(Y[:,c]==1)[0]
    plt.scatter(X[idx,0], X[idx,1], s=1)
plt.show()



## Split into Train, Validatio and Test

In [ ]:
num_train_samples = math.floor(num_samples*0.7)
num_validation_samples = math.floor(num_samples*0.2)
num_test_samples = num_samples - num_train_samples - num_validation_samples

X_train, Y_train = X[:num_train_samples], Y[:num_train_samples]
X_validation, Y_validation = X[num_train_samples:num_train_samples+num_validation_samples], Y[num_train_samples:num_train_samples+num_validation_samples]
X_test, Y_test = X[num_train_samples+num_validation_samples:], Y[num_train_samples+num_validation_samples:]

print(X_train.shape, Y_train.shape)
print(X_validation.shape, Y_validation.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
epochs = 100
batch_size = 32
learning_rate = 0.01


model = torch.nn.Sequential(
    torch.nn.Linear(2, 10),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(10, num_classes),
    torch.nn.Softmax(dim=1)
)
model.to(device)

# https://pytorch.org/docs/stable/nn.html#loss-functions
# loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_losses = []
validation_losses = []

for epoch in range(epochs):
    model.train()
    loss_val = 0
    for i in range(0, num_train_samples, batch_size):
        X_batch = X_train[i:i+batch_size].to(device)
        Y_batch = Y_train[i:i+batch_size].to(device)

        optimizer.zero_grad()
        Y_pred = model(X_batch)
        loss = loss_fn(Y_pred, Y_batch)
        loss.backward()
        optimizer.step()

        loss_val += loss.item()*len(X_batch)

    train_losses.append(loss_val/num_train_samples)

    model.eval()
    with torch.no_grad():
        Y_pred = model(X_validation)
        loss = loss_fn(Y_pred, Y_validation)

        validation_losses.append(loss.item())

    print(f"Epoch {epoch}, loss: {loss.item()}")

plt.figure()
plt.plot(train_losses, label="train")
plt.plot(validation_losses, label="validation")
plt.legend()
plt.show()

# model.eval()
# with torch.no_grad():
#     Y_pred = model(X_test)
#     loss = criterion(Y_pred, Y_test)
#     print(f"Test loss: {loss.item()}")

#     Y_pred = torch.argmax(Y_pred, dim=1)
#     Y_test = torch.argmax(Y_test, dim=1)
#     accuracy = torch.sum(Y_pred == Y_test).item() / num_test_samples
#     print(f"Test accuracy: {accuracy}")

#     plt.figure()
#     for c in range(num_classes):
#         idx = torch.where(Y_test==c)[0]
#         plt.scatter(X_test[idx,0], X_test[idx,1], s=1, c=Y_pred[idx].numpy())
#     plt.show(
# )